# CliffordLab Tutorial

A hands-on, well-documented notebook to experiment with **unified slice regularity** on real Clifford algebras $\\mathrm{Cl}(p,q)$.

**What you'll do here**:
1. Quick start with the pure-Python core (`cliffordlab.py`).
2. Explore unified slices ($J^2=\\pm1$), check **CR conditions** numerically.
3. Use **SymPy** for exact, symbolic CR checks and closed-form identities.
4. Try a simple **Cauchy-type reconstruction** on both elliptic and hyperbolic slices.
5. (Optional) Run the **pygae/clifford** backend.

> Tip: cells are numbered and independent; feel free to jump around.


## 0. Setup (optional installs)
You can run the core without extra dependencies. For the symbolic and GA backends:
```bash
pip install sympy clifford
```
Make sure this notebook is in the same folder as:
- `cliffordlab.py`
- `cliffordlab_sym.py`
- `cliffordlab_pygae.py`


## 1. Quick Start (core lab)
Let's run the minimal lab and see the output for the exponential example on both slices.

In [ ]:
import json
import cliffordlab as CL

out = CL.demo_basic()
print(json.dumps(out, indent=2))

**What to expect**

- `CR_residuals` should be small numbers (close to 0) for both slices.
- `Cauchy_reconstruction` reports a simple numerical reconstruction for $f(z_0)$ on each slice.
  The hyperbolic case is intentionally basic; accuracy improves with better contours/quadrature.

## 2. Unified slices and CR conditions (numeric)
We define a test function via the slice power series, then measure CR residuals at a point.

In [ ]:
from math import pi
def fAB(x, y, sign):
    return CL.slice_exp((x, y), sign, N=40)

for sign, name in [(-1, 'elliptic (J^2=-1)'), (+1, 'hyperbolic (J^2=+1)')]:
    r1, r2 = CL.CR_residuals(fAB, 0.2, 0.3, sign)
    print(name, 'CR residuals:', r1, r2)

## 3. Symbolic mode with SymPy
Check CR conditions **exactly** and verify the closed forms for the slice exponential.

In [ ]:
import sympy as sp
import cliffordlab_sym as SYM

# Series-based symbolic CR check
(x, y), (A, B), (r1, r2) = SYM.example_exp_symbolic(sign=-1, seriesN=40)
print('Elliptic series CR residuals (symbolic):', sp.simplify(r1), sp.simplify(r2))
(x, y), (A, B), (r1, r2) = SYM.example_exp_symbolic(sign=+1, seriesN=40)
print('Hyperbolic series CR residuals (symbolic):', sp.simplify(r1), sp.simplify(r2))

# Closed-form identities
(x, y), (Ae, Be, r1e, r2e), (Ah, Bh, r1h, r2h) = SYM.example_exp_closed_forms()
print('Closed-form CR (elliptic):', sp.simplify(r1e), sp.simplify(r2e))
print('Closed-form CR (hyperbolic):', sp.simplify(r1h), sp.simplify(r2h))

## 4. Cauchy-type reconstruction: accuracy vs. N
We reconstruct $f(z_0)$ from boundary data on the slice. For the elliptic case we use a circle; for the hyperbolic case we use a spacelike rectangle. Increase `N` to observe convergence.

In [ ]:
# Measure reconstruction error as a function of N
import numpy as np
import matplotlib.pyplot as plt

z0 = (0.2, 0.3)
Ns = [100, 200, 400, 800, 1200]
errs_e = []
errs_h = []
true_e = fAB(*z0, -1)
true_h = fAB(*z0, +1)
for N in Ns:
    rec_e = CL.cauchy_reconstruct(fAB, z0, -1, radius=0.35, N=N)
    rec_h = CL.cauchy_reconstruct(fAB, z0, +1, radius=0.35, N=N)
    errs_e.append(abs(rec_e[0]-true_e[0]) + abs(rec_e[1]-true_e[1]))
    errs_h.append(abs(rec_h[0]-true_h[0]) + abs(rec_h[1]-true_h[1]))
plt.figure()
plt.plot(Ns, errs_e, marker='o', label='elliptic (J^2=-1)')
plt.plot(Ns, errs_h, marker='s', label='hyperbolic (J^2=+1)')
plt.xlabel('N (panels)')
plt.ylabel('L1 error of reconstruction')
plt.title('Cauchy reconstruction error vs N')
plt.legend()
plt.show()

**Notes**
- Elliptic (circle) typically converges faster.
- Hyperbolic (spacelike rectangle) is sensitive to the null set $x^2 - y^2 = 0$; more careful contours and Gaussian quadrature improve accuracy.

## 5. (Optional) pygae/clifford backend
If you have `clifford` installed, you can validate $J^2$ inside $\\mathrm{Cl}(p,q)$ and reuse the same slice functions.

In [ ]:
try:
    import cliffordlab_pygae as CGA
    from pprint import pprint
    pprint(CGA.example_exp_Cl31())
except Exception as e:
    print('pygae/clifford not installed or unavailable:', e)